In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.preprocessing import LabelEncoder
import numpy as np
import folium
from dateutil.parser import parse

# Load data
data = pd.read_csv(r"C:\Users\danas\Downloads\Madis_Locations (2).csv")

# Function to parse dates with a fallback to infer format
def parse_date(date_str):
    try:
        return pd.to_datetime(date_str, format='%m/%d/%Y')
    except ValueError:
        return pd.to_datetime(date_str, errors='coerce')

# Convert 'Date' and 'Arrival Time' to datetime
data['Date'] = data['Date'].apply(parse_date)
data['Arrival Time'] = pd.to_datetime(data['Arrival Time'], format='%I:%M %p', errors='coerce')

# Drop rows with missing 'Date', 'Arrival Time', 'lat', 'lon', or 'High Temp'
data = data.dropna(subset=['Date', 'Arrival Time', 'lat', 'lon', 'high temp'])

# Extract additional features
data['DOW'] = data['Date'].dt.dayofweek
data['Hour'] = data['Arrival Time'].dt.hour

# Combine lat and lon to create a location identifier
data['LocationID'] = data['lat'].astype(str) + ',' + data['lon'].astype(str)

# Define features and target
X = data[['DOW', 'Hour', 'high temp']]
y = data['LocationID']

# Encode target labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train the model
location_model = RandomForestClassifier()
location_model.fit(X_train, y_train)

# Predict on the test set
y_pred = location_model.predict(X_test)

# Decoding predicted and actual labels back to LocationID
y_test_decoded = le.inverse_transform(y_test)
y_pred_decoded = le.inverse_transform(y_pred)

# Calculating MSE for latitude and longitude
lat_test = np.array([float(loc.split(',')[0]) for loc in y_test_decoded])
lon_test = np.array([float(loc.split(',')[1]) for loc in y_test_decoded])
lat_pred = np.array([float(loc.split(',')[0]) for loc in y_pred_decoded])
lon_pred = np.array([float(loc.split(',')[1]) for loc in y_pred_decoded])

lat_mse = mean_squared_error(lat_test, lat_pred)
lon_mse = mean_squared_error(lon_test, lon_pred)

print(f'Latitude MSE: {lat_mse}')
print(f'Longitude MSE: {lon_mse}')

# Prediction function
def predict_location(day_name, time_str, temp):
    dow_mapping = {
        'Monday': 0,
        'Tuesday': 1,
        'Wednesday': 2,
        'Thursday': 3,
        'Friday': 4,
        'Saturday': 5,
        'Sunday': 6
    }
    dow = dow_mapping.get(day_name.title())
    if dow is None:
        print("Invalid day name entered.")
        return None, None, None
    parsed_time = parse(time_str)
    if parsed_time is None:
        print("Invalid time format entered.")
        return None, None, None
    hour = parsed_time.hour
    input_data = pd.DataFrame([[dow, hour, temp]], columns=['DOW', 'Hour', 'high temp'])
    location_id_pred = location_model.predict(input_data)[0]
    location_id_pred_decoded = le.inverse_transform([location_id_pred])[0]
    lat_pred, lon_pred = map(float, location_id_pred_decoded.split(','))
    address_pred = data[data['LocationID'] == location_id_pred_decoded]['Location'].iloc[0]
    return lat_pred, lon_pred, address_pred

# Function to display the map
def display_map(lat, lon, address):
    # Create a map centered around the predicted location
    m = folium.Map(location=[lat, lon], zoom_start=20)
    # Add a marker for the predicted location
    folium.Marker([lat, lon], popup=address).add_to(m)
    # Save the map as an HTML file and display it
    map_filename = f'predicted_location_at_{address}.html'
    m.save(map_filename)
    return map_filename

# User inputs for day of the week, hour, and temperature
print(f'Enter the day of the week (e.g., Monday, Tuesday, ...): ')
day_name = input()
print("Enter the time (e.g., 4:45 pm, 1:00 am): ")
time_str = input()
print("Enter the high temperature of the day in Fahrenheit: ")
temp = float(input())

pred_lat, pred_lon, pred_address = predict_location(day_name, time_str, temp)
print(f'On {day_name}, at {time_str}, the predicted location is ({pred_lat}, {pred_lon}), {pred_address}.')

map_filename = display_map(pred_lat, pred_lon, pred_address)


import pandas as pd
from dateutil.parser import parse

# Load the schedule data
schedule_data = pd.read_csv(r"C:\Users\danas\Downloads\Madis Schedule.csv")

# Function to parse time in various formats
def parse_time(time_str):
    try:
        return parse(time_str).time()
    except ValueError:
        return None

# Function to check if the given time is within the time range
def is_within_time_range(time, start_time, end_time):
    if start_time <= time <= end_time:
        return True
    # For time ranges that span midnight
    if end_time < start_time:
        return time >= start_time or time <= end_time
    return False

specified_locations = ['Mackenzie', 'Gerlinger', 'Condon', 'Klamath']
# Function to check if Madi is scheduled to be at a given location at a given time
def is_scheduled_location(day_name, time_str, location_data):
    # Convert day name to lowercase for consistency
    day_name = day_name.lower()
    # Parse the input time
    input_time = parse_time(time_str)
    if input_time is None:
        print("Invalid time format entered.")
        return False
    for index, row in location_data.iterrows():
        # Check if the input day matches the schedule day
        if row['DOW'].lower() == day_name:
            start_time = parse_time(row['Arrival Time'])
            end_time = parse_time(row['Departure Time'])
            # Check if the input time falls within the time range of the schedule
            if is_within_time_range(input_time, start_time, end_time):
                if row['Location'] in specified_locations and pred_address != row['Location']:
                    print(f"Madi is likely skipping class.")
                    return True

# Check if Madi is scheduled to be at the predicted location at the given time
is_scheduled_location(day_name, time_str, schedule_data)


